In [ ]:
import sympy as sp
from IPython.display import display
import inspect


class Errorpropagator:
    def __init__(self, func, name='func'):
        self.name = name
        self.F = sp.Symbol('f')
        self.f = sp.parse_expr(func)
        symbols = list(map(str, list(self.f.free_symbols)))
        self.x__ = sp.Symbol('x')
        self.symbols = []
        for symbol in symbols:
            self.symbols.append(
                [sp.Symbol(f'{symbol}'), sp.Symbol('\Delta ' + symbol)])

        df_sim = 0
        df = 0
        for i, di in self.symbols:
            df_sim += sp.simplify((sp.diff(self.f, i)*di/self.f)**2)
            df += (sp.diff(self.f, i)*di)**2

        self.df_sim = self.F*sp.sqrt(df_sim)
        for i in range(3):
            self.df_sim = sp.simplify(self.df_sim)
        self.latex_sim = sp.latex(self.df_sim)

        self.df = sp.sqrt(df)
        self.latex = sp.latex(self.df)

        ### get source code ###
        self.symbols_d = []
        for symbol in symbols:
            self.symbols_d.append(
                [sp.Symbol(f'{symbol}'), sp.Symbol('d' + symbol)])

        df = 0
        for i, di in self.symbols_d:
            df += (sp.diff(self.f, i)*di)**2
        self.df_d = sp.sqrt(df)

    def showme(self):
        class Code:
            def __init__(self, x_, f_):
                raw_code = inspect.getsource(
                    sp.lambdify(x_, f_)).split('return')[-1]
                raw_code = raw_code.replace('exp', 'np.exp')
                raw_code = raw_code.replace('sqrt', 'np.sqrt')
                raw_code = raw_code.replace('log', 'np.log')
                raw_code = raw_code.replace('ln', 'np.log')

                raw_code = raw_code.replace('sin(', 'np.sin(')
                raw_code = raw_code.replace('cos(', 'np.cos(')
                raw_code = raw_code.replace('tan(', 'np.tan(')

                raw_code = raw_code.replace('sinh(', 'np.sinh(')
                raw_code = raw_code.replace('cosh(', 'np.cosh(')
                raw_code = raw_code.replace('tanh(', 'np.tanh(')

                raw_code = raw_code.replace('atan(', 'np.arctan(')
                raw_code = raw_code.replace('asin(', 'np.arcsin(')
                raw_code = raw_code.replace('acos(', 'np.arccos(')

                raw_code = raw_code.replace('atanh(', 'np.arctanh(')
                raw_code = raw_code.replace('asinh(', 'np.arcsinh(')
                raw_code = raw_code.replace('acosh(', 'np.arccosh(')
                self.raw_code = raw_code
                t = sp.lambdify(x_, f_)

        self.free_f = sorted(list(map(str, list(self.f.free_symbols))))
        self.free_f = ['x'] + sorted([i for i in self.free_f if i != 'x'])
        self.free_df = list(map(str, list(self.df.free_symbols)))
        self.free_df = [i.replace('\\Delta ', 'd') for i in self.free_df]
        self.free_df1 = sorted(
            [i for i in self.free_df if 'd' not in i and i != 'x'])
        self.free_df2 = sorted(
            [i for i in self.free_df if 'd' in i and i != 'dx'])
        self.free_df = ['x']+self.free_df1+['dx']+self.free_df2

        self.free_df = ','.join(self.free_df)
        self.free_f = ','.join(self.free_f)

        n = 180
        print('-'+'#'*n+'-')
        print('input function:')
        print('\t', sp.latex(self.f))
        print('\njupyter only representation:')
        display(self.f)

        print('#'*n)

        print('LaTeX code of propagated error:')
        print('\t', sp.latex(self.df))
        print('\njupyter only representation:')
        display(self.df)

        print('#'*n)

        print('LaTeX code of simplified propagated error:')
        print('\t', sp.latex(self.df_sim))
        print('\njupyter only representation:')
        display(self.df_sim)

        print('#'*n)

        print('LaTeX code of simplified expanded propagated error:')
        print('\t', sp.latex(self.df_sim))
        print('\njupyter only representation:')
        display(self.df_sim.expand())

        print('#'*n)

        print('python code of input function:')
        print('def '+self.name+'('+self.free_f+'):')
        print('\treturn' + Code(self.x__, self.f).raw_code)
        print('python code of error function:')
        print('def d'+self.name+'('+self.free_df+'):')
        print('\treturn' + Code(self.x__, self.df_d).raw_code)


def main(expression, name='func'):
    '''
    this little script gives you the followings things:
        1) python code for the expression you can copy directly into your python script
        2) python code for the propagated error you can copy directly into your python script
        2) on top, you get various latex code for the expressions stated above
    params:
        :expression: enter your expression like you would in python. you can use <variable>_<index>, to index your variables. You can not use ^ 
        :name: optional, for easier copy and paste, give the function a name
        call Errorpropagator(<expression>).showme()
        translation table:
            np.arcsin -> asin
            np.sinh -> sinh


    '''
    E = Errorpropagator(expression, name)
    E.showme()
